# 🎯 Missile Fixed PID Optimization with RL - Kaggle GPU Training

Train RL agents to find optimal **FIXED** PID parameters (set once, stay constant).

**Difference from Adaptive PID:**
- **Fixed PID** (this notebook): RL learns optimal static PID values → practical, interpretable
- **Adaptive PID** (other notebook): RL adjusts PID every step → advanced, complex

**Requirements:**
- Kaggle GPU (T4 or P100)
- Internet enabled (for pip install)

**Training Time:** ~20-30 minutes (500K timesteps)

## 1. Setup Environment

In [ ]:
%%time
# Install dependencies with fixed versions
!pip install -q 'numpy<2' gymnasium 'stable-baselines3[extra]' torch tensorboard

print("\n✓ Dependencies installed")

In [ ]:
# Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be slow.")

## 2. Define Missile PID System

In [ ]:
import numpy as np

class PIDController:
    """Simple PID controller"""
    def __init__(self, kp=2.0, ki=0.1, kd=0.5):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral = 0.0
        self.prev_error = 0.0

    def compute(self, error, dt):
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        output = self.kp * error + self.ki * self.integral + self.kd * derivative
        self.prev_error = error
        return output

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0

    def set_gains(self, kp, ki, kd):
        self.kp = kp
        self.ki = ki
        self.kd = kd


class Missile:
    """2D Missile with PID guidance"""
    def __init__(self, x=0.0, y=0.0, vx=800.0, vy=0.0,
                 max_speed=1000.0, max_accel=1000.0,
                 kp=2.0, ki=0.1, kd=0.5):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.max_speed = max_speed
        self.max_accel = max_accel
        self.pid = PIDController(kp, ki, kd)
        self.active = True
        self.fuel = 1.0

    @property
    def speed(self):
        return np.sqrt(self.vx**2 + self.vy**2)

    @property
    def heading(self):
        return np.arctan2(self.vy, self.vx)

    def update(self, target_pos, dt):
        if not self.active:
            return

        dx = target_pos[0] - self.x
        dy = target_pos[1] - self.y
        desired_heading = np.arctan2(dy, dx)
        error = desired_heading - self.heading
        error = np.arctan2(np.sin(error), np.cos(error))

        control = self.pid.compute(error, dt)
        control = np.clip(control, -self.max_accel, self.max_accel)

        if self.speed > 0:
            perp = np.array([-self.vy, self.vx]) / self.speed
            self.vx += perp[0] * control * dt
            self.vy += perp[1] * control * dt

        current_speed = self.speed
        if current_speed > self.max_speed:
            scale = self.max_speed / current_speed
            self.vx *= scale
            self.vy *= scale

        self.x += self.vx * dt
        self.y += self.vy * dt
        self.fuel -= 0.01 * dt
        
        if self.fuel <= 0:
            self.active = False

print("✓ Missile system defined")

## 3. Define Moving Target

In [ ]:
class Target:
    """Moving target with different maneuver patterns"""
    def __init__(self, x=8000.0, y=5000.0, speed=1000.0, maneuver='straight'):
        self.x = x
        self.y = y
        self.speed = speed
        self.heading = np.pi
        self.maneuver = maneuver
        self.time = 0.0

    @property
    def velocity(self):
        vx = self.speed * np.cos(self.heading)
        vy = self.speed * np.sin(self.heading)
        return np.array([vx, vy])

    def update(self, dt, missile_pos=None):
        self.time += dt

        if self.maneuver == 'straight':
            pass
        elif self.maneuver == 'circular':
            turn_rate = 0.3
            self.heading += turn_rate * dt
        elif self.maneuver == 'zigzag':
            if int(self.time) % 4 < 2:
                turn_rate = 0.5
            else:
                turn_rate = -0.5
            self.heading += turn_rate * dt
        elif self.maneuver == 'evasive' and missile_pos is not None:
            dx = self.x - missile_pos[0]
            dy = self.y - missile_pos[1]
            distance = np.sqrt(dx**2 + dy**2)
            if distance < 3000:
                escape_heading = np.arctan2(dy, dx)
                heading_diff = escape_heading - self.heading
                heading_diff = np.arctan2(np.sin(heading_diff), np.cos(heading_diff))
                turn_rate = 0.8 * np.sign(heading_diff)
                self.heading += turn_rate * dt

        vx = self.speed * np.cos(self.heading)
        vy = self.speed * np.sin(self.heading)
        self.x += vx * dt
        self.y += vy * dt

print("✓ Target system defined")

## 4. Define Fixed PID Environment ⭐

**Key difference:** Agent sets PID parameters ONCE at episode start, then they stay FIXED.

In [ ]:
import gymnasium as gym
from gymnasium import spaces

class FixedPIDEnv(gym.Env):
    """Environment for learning optimal FIXED PID parameters"""
    metadata = {'render_modes': ['human', 'rgb_array'], 'render_fps': 60}

    def __init__(self, map_size=10000.0, hit_radius=50.0, max_steps=500,
                 dt=0.01, target_maneuver='circular',
                 missile_speed=1000.0, missile_accel=1000.0, target_speed=1000.0):
        super().__init__()

        # Action: Direct PID parameter values (set once per episode)
        self.action_space = spaces.Box(
            low=np.array([0.1, 0.0, 0.0], dtype=np.float32),
            high=np.array([10.0, 2.0, 5.0], dtype=np.float32),
            dtype=np.float32
        )

        # Observation: No PID values needed (they're fixed)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(11,), dtype=np.float32
        )

        self.map_size = map_size
        self.hit_radius = hit_radius
        self.max_steps = max_steps
        self.dt = dt
        self.target_maneuver = target_maneuver
        self.missile_speed = missile_speed
        self.missile_accel = missile_accel
        self.target_speed = target_speed

        self.missile = None
        self.target = None
        self.steps = 0
        self.fixed_pid_set = False

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        initial_vx = np.random.uniform(0.8 * self.missile_speed, 0.9 * self.missile_speed)
        self.missile = Missile(
            x=0.0, y=0.0, vx=initial_vx, vy=0.0,
            max_speed=self.missile_speed,
            max_accel=self.missile_accel,
            kp=2.0, ki=0.1, kd=0.5
        )

        target_x = np.random.uniform(0.6 * self.map_size, 0.9 * self.map_size)
        target_y = np.random.uniform(0.3 * self.map_size, 0.7 * self.map_size)
        self.target = Target(
            x=target_x, y=target_y,
            speed=self.target_speed,
            maneuver=self.target_maneuver
        )

        self.steps = 0
        self.fixed_pid_set = False

        return self._get_obs(), {}

    def step(self, action):
        # On FIRST step: set PID parameters from action
        if not self.fixed_pid_set:
            kp = float(action[0])
            ki = float(action[1])
            kd = float(action[2])
            self.missile.pid.kp = kp
            self.missile.pid.ki = ki
            self.missile.pid.kd = kd
            self.fixed_pid_set = True

        # On subsequent steps: action is IGNORED (PID stays fixed)

        # Update simulation
        self.missile.update(np.array([self.target.x, self.target.y]), self.dt)
        self.target.update(self.dt, missile_pos=np.array([self.missile.x, self.missile.y]))
        self.steps += 1

        # Calculate distance
        dx = self.target.x - self.missile.x
        dy = self.target.y - self.missile.y
        distance = np.sqrt(dx**2 + dy**2)

        # Check termination
        hit = distance < self.hit_radius
        out_of_bounds = (self.missile.x < -1000 or self.missile.x > self.map_size + 1000 or
                        self.missile.y < -1000 or self.missile.y > self.map_size + 1000)
        out_of_fuel = self.missile.fuel <= 0
        max_steps_reached = self.steps >= self.max_steps

        done = hit or out_of_bounds or out_of_fuel or max_steps_reached

        # Reward calculation
        reward = -distance / 1000.0

        if hit:
            reward += 100.0
            time_bonus = (self.max_steps - self.steps) / 10.0
            reward += time_bonus

        if (out_of_bounds or out_of_fuel or max_steps_reached) and not hit:
            reward -= 50.0

        if self.missile.fuel > 0:
            reward += self.missile.fuel / 10000.0

        info = {
            'hit': hit,
            'distance': distance,
            'steps': self.steps,
            'pid_kp': self.missile.pid.kp,
            'pid_ki': self.missile.pid.ki,
            'pid_kd': self.missile.pid.kd,
            'fuel': self.missile.fuel
        }

        return self._get_obs(), reward, done, False, info

    def _get_obs(self):
        # Missile state
        m_x = self.missile.x / self.map_size
        m_y = self.missile.y / self.map_size
        m_vx = self.missile.vx / self.missile.max_speed
        m_vy = self.missile.vy / self.missile.max_speed

        # Target state
        t_x = self.target.x / self.map_size
        t_y = self.target.y / self.map_size
        target_vel = self.target.velocity
        t_vx = target_vel[0] / self.target.speed
        t_vy = target_vel[1] / self.target.speed

        # Relative info
        dx = self.target.x - self.missile.x
        dy = self.target.y - self.missile.y
        distance = np.sqrt(dx**2 + dy**2) / self.map_size

        # Angle error
        desired_angle = np.arctan2(dy, dx)
        current_angle = np.arctan2(self.missile.vy, self.missile.vx)
        angle_error = desired_angle - current_angle
        angle_error = np.arctan2(np.sin(angle_error), np.cos(angle_error))

        # Fuel
        fuel = self.missile.fuel / 1.0

        obs = np.array([
            m_x, m_y, m_vx, m_vy,
            t_x, t_y, t_vx, t_vy,
            distance, angle_error, fuel
        ], dtype=np.float32)

        return obs

print("✓ Fixed PID environment defined")

## 5. Configuration

In [ ]:
# Training configuration
ALGORITHM = 'SAC'  # SAC works best for this task
TARGET_MANEUVER = 'circular'  # straight, circular, zigzag, evasive
TOTAL_TIMESTEPS = 500_000  # 500K timesteps
N_ENVS = 1

# Speed configuration
MISSILE_SPEED = 1000.0  # m/s
MISSILE_ACCEL = 1000.0  # m/s²
TARGET_SPEED = 1000.0   # m/s

print(f"Configuration:")
print(f"  Algorithm: {ALGORITHM}")
print(f"  Target Maneuver: {TARGET_MANEUVER}")
print(f"  Total Timesteps: {TOTAL_TIMESTEPS:,}")
print(f"\n🚀 Speed Config:")
print(f"  Missile: {MISSILE_SPEED} m/s, {MISSILE_ACCEL} m/s²")
print(f"  Target: {TARGET_SPEED} m/s")
print(f"\n🎯 Goal: Find optimal FIXED PID parameters")

## 6. Create Training Environment

In [ ]:
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
import os
from datetime import datetime

def make_env(rank):
    def _init():
        return FixedPIDEnv(
            target_maneuver=TARGET_MANEUVER,
            missile_speed=MISSILE_SPEED,
            missile_accel=MISSILE_ACCEL,
            target_speed=TARGET_SPEED
        )
    return _init

env = DummyVecEnv([make_env(0)])
eval_env = DummyVecEnv([make_env(0)])

print("✓ Training environment created")

## 7. Initialize RL Algorithm

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"FIXED_{ALGORITHM}_{TARGET_MANEUVER}_{timestamp}"
run_dir = f"/kaggle/working/{run_name}"
os.makedirs(run_dir, exist_ok=True)

# Deep network architecture
policy_kwargs = dict(
    net_arch=dict(
        pi=[256, 256, 256],
        qf=[256, 256, 256]
    )
)

if ALGORITHM == 'SAC':
    model = SAC(
        'MlpPolicy',
        env,
        learning_rate=3e-4,
        buffer_size=100_000,
        learning_starts=1000,
        batch_size=256,
        tau=0.005,
        gamma=0.99,
        train_freq=1,
        gradient_steps=1,
        verbose=1,
        tensorboard_log=os.path.join(run_dir, 'tensorboard'),
        device='cuda',
        policy_kwargs=policy_kwargs
    )
elif ALGORITHM == 'PPO':
    policy_kwargs['net_arch'] = [256, 256, 256]
    model = PPO(
        'MlpPolicy',
        env,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        verbose=1,
        tensorboard_log=os.path.join(run_dir, 'tensorboard'),
        device='cuda',
        policy_kwargs=policy_kwargs
    )

print(f"✓ {ALGORITHM} model initialized")
print(f"✓ Output: {run_dir}")

## 8. Setup Callbacks

In [ ]:
checkpoint_callback = CheckpointCallback(
    save_freq=25_000,
    save_path=os.path.join(run_dir, 'checkpoints'),
    name_prefix='model'
)

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=os.path.join(run_dir, 'best_model'),
    log_path=os.path.join(run_dir, 'eval_logs'),
    eval_freq=10_000,
    n_eval_episodes=5,
    deterministic=True
)

print("✓ Callbacks configured")

## 9. Train Model 🚀

Training will take ~20-30 minutes.

In [ ]:
%%time
print(f"\n{'='*60}")
print(f"Training Fixed PID with {ALGORITHM}")
print(f"Target: {TARGET_MANEUVER}")
print(f"Timesteps: {TOTAL_TIMESTEPS:,}")
print(f"{'='*60}\n")

model.learn(
    total_timesteps=TOTAL_TIMESTEPS,
    callback=[checkpoint_callback, eval_callback],
    progress_bar=True
)

final_path = os.path.join(run_dir, 'final_model')
model.save(final_path)

print(f"\n{'='*60}")
print(f"✓ Training complete!")
print(f"✓ Model saved: {final_path}")
print(f"{'='*60}")

## 10. Extract Optimal PID Parameters ⭐

This is the main output: optimal PID values for the target maneuver!

In [ ]:
# Load best model
best_model_path = os.path.join(run_dir, 'best_model', 'best_model.zip')
if ALGORITHM == 'SAC':
    eval_model = SAC.load(best_model_path)
elif ALGORITHM == 'PPO':
    eval_model = PPO.load(best_model_path)

# Test and extract PID parameters
test_env = FixedPIDEnv(
    target_maneuver=TARGET_MANEUVER,
    missile_speed=MISSILE_SPEED,
    missile_accel=MISSILE_ACCEL,
    target_speed=TARGET_SPEED
)

n_test_episodes = 10
hits = 0
pid_values = []
rewards = []
distances = []

print(f"\n{'='*60}")
print(f"EXTRACTING OPTIMAL PID PARAMETERS")
print(f"{'='*60}\n")

for episode in range(n_test_episodes):
    obs, _ = test_env.reset()
    episode_reward = 0
    done = False

    while not done:
        action, _ = eval_model.predict(obs, deterministic=True)
        obs, reward, done, _, info = test_env.step(action)
        episode_reward += reward

    rewards.append(episode_reward)
    distances.append(info['distance'])
    
    if info['hit']:
        hits += 1
    
    pid_values.append([info['pid_kp'], info['pid_ki'], info['pid_kd']])
    
    print(f"Episode {episode+1}: {'HIT' if info['hit'] else 'MISS':4s} | "
          f"Dist={info['distance']:6.1f}m | "
          f"Kp={info['pid_kp']:.3f}, Ki={info['pid_ki']:.3f}, Kd={info['pid_kd']:.3f}")

# Calculate optimal PID values
pid_values = np.array(pid_values)
optimal_kp = np.mean(pid_values[:, 0])
optimal_ki = np.mean(pid_values[:, 1])
optimal_kd = np.mean(pid_values[:, 2])
std_kp = np.std(pid_values[:, 0])
std_ki = np.std(pid_values[:, 1])
std_kd = np.std(pid_values[:, 2])

print(f"\n{'='*60}")
print(f"OPTIMAL PID PARAMETERS FOR '{TARGET_MANEUVER}' TARGET")
print(f"{'='*60}")
print(f"  Kp = {optimal_kp:.3f} ± {std_kp:.3f}")
print(f"  Ki = {optimal_ki:.3f} ± {std_ki:.3f}")
print(f"  Kd = {optimal_kd:.3f} ± {std_kd:.3f}")
print(f"\n  Hit Rate: {hits}/{n_test_episodes} ({100*hits/n_test_episodes:.1f}%)")
print(f"  Avg Reward: {np.mean(rewards):.1f}")
print(f"  Avg Distance: {np.mean(distances):.1f}m")
print(f"{'='*60}")

print(f"\n💡 Use these values in your missile system:")
print(f"\n   missile.set_pid_gains(kp={optimal_kp:.3f}, ki={optimal_ki:.3f}, kd={optimal_kd:.3f})")
print(f"\n   Or test in demo.py:")
print(f"   python demo.py --maneuver {TARGET_MANEUVER} --kp {optimal_kp:.3f} --ki {optimal_ki:.3f} --kd {optimal_kd:.3f}\n")

## 11. Visualize Results

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# PID parameters distribution
axes[0].hist(pid_values[:, 0], bins=10, alpha=0.7, color='red', edgecolor='black')
axes[0].axvline(optimal_kp, color='darkred', linestyle='--', linewidth=2, label=f'Mean: {optimal_kp:.3f}')
axes[0].set_xlabel('Kp', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Proportional Gain Distribution', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].hist(pid_values[:, 1], bins=10, alpha=0.7, color='green', edgecolor='black')
axes[1].axvline(optimal_ki, color='darkgreen', linestyle='--', linewidth=2, label=f'Mean: {optimal_ki:.3f}')
axes[1].set_xlabel('Ki', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Integral Gain Distribution', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

axes[2].hist(pid_values[:, 2], bins=10, alpha=0.7, color='blue', edgecolor='black')
axes[2].axvline(optimal_kd, color='darkblue', linestyle='--', linewidth=2, label=f'Mean: {optimal_kd:.3f}')
axes[2].set_xlabel('Kd', fontsize=12)
axes[2].set_ylabel('Frequency', fontsize=12)
axes[2].set_title('Derivative Gain Distribution', fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(run_dir, 'pid_distribution.png'), dpi=150)
plt.show()

print(f"✓ PID distribution plot saved")

## 12. Download Trained Model

In [ ]:
import shutil

# Save optimal PID values to text file
with open(os.path.join(run_dir, 'optimal_pid.txt'), 'w') as f:
    f.write(f"Optimal PID Parameters for '{TARGET_MANEUVER}' target\n")
    f.write(f"="*50 + "\n\n")
    f.write(f"Kp = {optimal_kp:.3f} ± {std_kp:.3f}\n")
    f.write(f"Ki = {optimal_ki:.3f} ± {std_ki:.3f}\n")
    f.write(f"Kd = {optimal_kd:.3f} ± {std_kd:.3f}\n\n")
    f.write(f"Hit Rate: {hits}/{n_test_episodes} ({100*hits/n_test_episodes:.1f}%)\n")
    f.write(f"Avg Reward: {np.mean(rewards):.1f}\n")
    f.write(f"Avg Distance: {np.mean(distances):.1f}m\n")

archive_name = f"{run_name}_model"
shutil.make_archive(
    f"/kaggle/working/{archive_name}",
    'zip',
    run_dir
)

print(f"✓ Model archived: /kaggle/working/{archive_name}.zip")
print(f"📥 Download from Kaggle Output panel")
print(f"\nContents:")
print(f"  - optimal_pid.txt     (Optimal PID values)")
print(f"  - best_model/         (Trained model)")
print(f"  - pid_distribution.png (Visualization)")

## 13. Summary

### What we did:
1. ✅ Set up GPU environment
2. ✅ Defined Fixed PID environment
3. ✅ Trained RL agent to find optimal FIXED PID parameters
4. ✅ Extracted optimal PID values
5. ✅ Visualized results

### Key Results:
- **Optimal Kp, Ki, Kd** values for your target maneuver
- Ready to use in real missile system
- Interpretable and practical

### Next Steps:
1. Test these PID values in `demo.py`
2. Try different target maneuvers (straight, zigzag, evasive)
3. Compare with hand-tuned PID values
4. Deploy in your missile guidance system

### Why Fixed PID is Better:
- ✅ Practical: Real systems use fixed PID
- ✅ Interpretable: Clear PID values you can understand
- ✅ Fast: Learns quicker than adaptive approach
- ✅ Testable: Easy to validate and tune further